# Workshop Tutorial: Constraints

In this tutorial we will learn about constraints between Parameters and the default parameterization of a binary system.

This interactive workshop tutorial covers many of the same topics as the corresponding online tutorials:

* [Constraints](http://phoebe-project.org/docs/2.3/tutorials/constraints.ipynb)
* [Advanced: Constraints and Changing Hierarchies](http://phoebe-project.org/docs/2.3/tutorials/constraints_hierarchies.ipynb)
* [Advanced: Built-In Constraints](http://phoebe-project.org/docs/2.3/tutorials/constraints_builtin.ipynb)

# Setup

In [ ]:
import phoebe
from phoebe import u,c

In [ ]:
logger = phoebe.logger(clevel='WARNING')

In [ ]:
b = phoebe.default_binary()

# Constraints

As we saw in the last exercise, there are 5 Parameters with a qualifier of 'incl'.

In [ ]:
print(b.filter(qualifier='incl'))

Note that here the previously-mentioned twig-syntax is shown to show as much information as possible about the Parameters.

Three of these are because there are inclinations defined for the orbit as well as each of the two stars ('primary' and 'secondary').  These three Parameters all have context='component'.

In [ ]:
print(b.filter(qualifier='incl', context='component'))

The other inclinations of the stars are (by default) *constrained* to be the same as the inclination of the orbit (i.e., an aligned system).  We can see this by the \*s in the output (to the left of the twigs) above as well as by accessing the 'constrained_by' attribute of the Parameter (attempting to call set_value will also raise an error).

In [ ]:
b.get_parameter(qualifier='incl', context='component', component='primary').constrained_by

The other two Parameters with qualifier='incl' are the constraints themselves and have context='constraint'

In [ ]:
print(b.filter(qualifier='incl', context='constraint'))

In [ ]:
b.get_parameter(qualifier='incl', context='constraint', component='primary')

Here we see that this is a simple constraint: the inclination of the primary star is being *constrained* to be exactly that of the inclination of the binary orbit.  If we change the inclination of the orbit, the inclinations of the 'primary' and 'secondary' stars will immediately update to reflect that change.

In [ ]:
b.set_value(qualifier='incl', component='binary', value=80)

In [ ]:
print(b.filter(qualifier='incl', context='component'))

Other constraints are a little more complicated.

In [ ]:
b.get_parameter(qualifier='asini', context='constraint')

In [ ]:
print("asini: {}, sma: {}, incl: {}".format(
    b.get_value(qualifier='asini', component='binary', context='component'),
    b.get_value(qualifier='sma', component='binary', context='component'),
    b.get_value(qualifier='incl', component='binary', context='component')))

In [ ]:
b.set_value(qualifier='sma', component='binary', context='component', value=10.0)

In [ ]:
print("asini: {}, sma: {}, incl: {}".format(
    b.get_value(qualifier='asini', component='binary', context='component'),
    b.get_value(qualifier='sma', component='binary', context='component'),
    b.get_value(qualifier='incl', component='binary', context='component')))

# Exercise

Find and list all constraints in the default_binary Bundle

The mass of each star is a constrained Parameter.  Try setting the value of the mass (you should get an error, do **not** try to change the value of the constraint itself).  Then find what it depends on, vary those Parameters, and print the resulting value of the mass.

How q is defined: is it Mprimary/Msecondary or Msecondary/Mprimary?

Hint: there are (at least) 2 ways to do this.  Try first by looking through the equations of the constraints if you feel a bit daring.  You can also change the value of q and see how the resulting constrained masses react.

# Re-Parameterizing or "Flipping" Constraints

In [ ]:
b = phoebe.default_binary()

In the default binary, there are a significant number of constrained Parameters.

In [ ]:
print(b.filter(context='constraint').qualifiers)

Let's look at mass, which is _constrained_ by default according to Kepler's third law.

In [ ]:
print(b.get_parameter('mass', component='primary', context='component'))

In [ ]:
print(b.get_parameter('mass', component='primary', context='constraint'))

Here we see the 4 parameters that are involved in Kepler's third law. PHOEBE allows you to freely set 3 of these 4 (sma, period, q) and automatically uses these values to compute that mass.

However, let's say that you wanted to set the mass (perhaps you know the mass, but don't know the semi-major axis as well). This can be done via the 'flip_constraint' method. The easiest way to use this correctly is to make sure our keywords return the correct Constraint Parameter via 'get_constraint' and then use 'flip_constraint'.

In [ ]:
b.get_constraint(qualifier='mass', component='primary')

Now we just add `solve_for='sma'` to "flip" this constraint to solve for 'sma' instead of 'mass'.

In [ ]:
b.flip_constraint(qualifier='mass', component='primary', solve_for='sma')

Now we're allowed to set the mass and we'll see that the value of sma is automatically computed.

In [ ]:
b.set_value('mass', component='primary', value=1.2)

In [ ]:
b.get_value('sma', component='binary', context='component')

# Exercise

Flipping constraints could be particularly useful if you have an observational constraint on 'asini' (say from the amplitude of RVs) and want to leave asini fixed as you fit for the inclination.  Flip the constraint so it is possible to adjust the values of both 'asini' and 'incl'.  (**NOTE** you may want to either start fresh or re-flip the Kepler's third law constraint back to solve for mass first).

Now that you can change the value of 'asini', set it to 20 (solar radii... we'll talk about units in the next tutorial), adjust the inclination, and show that 'sma' is adjusting automatically to conserve 'asini'.